In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Introduction

The data set refers to clients of a wholesale distributor. It includes the annual spending in monetary units on diverse product categories The dataset is available at UCI machine learning repository.

Data Set Information:

Provide all relevant information about your data set.


### Attribute Information:

1) FRESH: annual spending on fresh products (Continuous);
2) MILK: annual spending on milk products (Continuous);
3) GROCERY: annual spending on grocery products (Continuous);
4) FROZEN: annual spending on frozen products (Continuous)
5) DETERGENTS_PAPER: annual spending on detergents and paper products (Continuous)
6) DELICATESSEN: annual spending on and delicatessen products (Continuous);
7) CHANNEL: customers™ Channel - Horeca (Hotel/Restaurant/Cafe) or Retail channel (Nominal)
8) REGION: customers™ Region“ Lisnon, Oporto or Other (Nominal)

### Descriptive Statistics:

(Minimum, Maximum, Mean, Std. Deviation)
FRESH ( 3, 112151, 12000.30, 12647.329)
MILK (55, 73498, 5796.27, 7380.377)
GROCERY (3, 92780, 7951.28, 9503.163)
FROZEN (25, 60869, 3071.93, 4854.673)
DETERGENTS_PAPER (3, 40827, 2881.49, 4767.854)
DELICATESSEN (3, 47943, 1524.87, 2820.106)

REGION Frequency
Lisbon 77
Oporto 47
Other Region 316
Total 440

CHANNEL Frequency
Horeca 298
Retail 142
Total 440

### Implementation:

    EDA and any data cleaning
    Implemented Feature Scaling to Normalize the data(compare the histogram and KDE for MinMaxScaler and StandardScaler)
    Finding optimal number of features using RFECV and shown the plot between Number of features selected vs Cross validation score (used channel as target variable)
    Implemented KMeans Clustering for K=2 to K=15 and based on elbow method identify what is the optimum number of clusters
    Implemented PCA with number of original features to answer how much variance is explained by first 2 components and by first 4 components and visualize the clusters in the data
    Implemented XGBoost Classifier with 5 Fold CV and report the performance metrics

### Task:
Goal of this project is to best describe the variation in the different types of customers that a wholesale distributor interacts with.



In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt  # for plotting facilities
import seaborn as sns; 
sns.set(color_codes=True)
from sklearn import preprocessing
from sklearn.decomposition import PCA 

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/kaggle/input/wholesale-customers-data-set/Wholesale customers data.csv")

# EDA

## Shape of Data set

In [ ]:
df.shape

440 instances and 8 attributes 

##  Preview dataset 

In [ ]:
df.head()

In [ ]:
print("Channel unique values:",df['Channel'].unique())
print("Region unique values",df['Region'].unique())

We can see that Channel variable contains values as 1 and 2.

These two values classify the customers from two different channels as

1 for Horeca (Hotel/Retail/Café) customers and
2 for Retail channel (nominal) customers.

Region - 3 unique values
Lisnon, Oporto or Other (Nominal)

## Summary of dataset

In [ ]:
df.info()

We can see that no null value and all the data inputs are having numeric datatype

## Summary statistics of dataset 

In [ ]:
df.describe()

## Check missing value

In [ ]:
df.isnull().sum()

We can see that there are no missing values in the dataset. Thus, there is no need to use the dropna() function.

 6 continuous types of feature ('Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicassen')
 2 categoricals features ('Channel', 'Region')

In [ ]:
categorical_features = ['Channel', 'Region']
continuous_features = ['Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicassen']

### Channel Count

In [ ]:
df['Channel'].value_counts()

In [ ]:
def categorical_multi(i,j):
    pd.crosstab(df[i],df[j]).plot(kind='bar')
    plt.show()
    print(pd.crosstab(df[i],df[j]))

categorical_multi(i='Channel',j='Region')    

In [ ]:
df.corr()

In [ ]:
# Correlation analysis
corrMatt = df.corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

Looking at the plot above, there are a few pairs of features that exhibit some degree of correlation. They include:

#### Grocery and Detergents_Paper are highly correlated  - 0.92
#### Milk and Groceries - 0.73
#### Milk and Detergents_Paper - 0.66

## Outliers

In [ ]:
scaler = preprocessing.MinMaxScaler()
scaled = scaler.fit_transform(df)
scaled_data = pd.DataFrame(scaled, columns = [name for name in list(df)])
fig = plt.figure(figsize = (20,10))
scaled_data.boxplot(column=[name for name in list(scaled_data)], grid=False)

This shows that no outliers present in the dataset

In [ ]:
# No obvious trend in data set as sequence
%matplotlib inline
df.plot(figsize=(12,6), style='.');

Q. Implement Feature Scaling to Normalize the data(compare the histogram/KDE for MinMaxScaler 
and StandardScaler). Choose one of the Scaler to proceed ahead and provide reasoning as to 
why it was selected?

Some features in our data might have high-magnitude values (for example, annual salary), while others might have relatively low values (for example, the number of years worked at a company). Just because some data has smaller values does not mean it is less significant. So, to make sure our prediction does not vary because of different magnitudes of features in our data, we can perform feature scaling, standardization, or normalization



Implementing Scaling Using the
## Standard Scaler Method

In [ ]:
df1 = df.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scale = StandardScaler().fit_transform(df1)

scaled_frame = pd.DataFrame(std_scale, columns=df1.columns)

scaled_frame.head()

In [ ]:
scaled_frame.describe()

In [ ]:
pd.DataFrame(scaled_frame).plot(kind='kde')

In [ ]:
pd.DataFrame(scaled_frame).plot(kind='hist', bins=10)

Using the StandardScaler method, we have scaled the data into a uniform unit over all the columns. As you can see in the preceding table, the values of all the features have been converted into a uniform range of the same scale. Because of this, it becomes easier for the model to make predictions.
Kind kde is seems to be better then hist because in hist, data seems to be overlapped and unable to see the value for each field, whereas for kde, data visualization is finer.

 Implementing Scaling Using the 
 ## MinMax Scaler Method

In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax_scale = MinMaxScaler().fit_transform(df)

scaled_frame2 = pd.DataFrame(minmax_scale,columns=df.columns)

scaled_frame2.head()

In [ ]:
scaled_frame2.describe()

In [ ]:
pd.DataFrame(scaled_frame2).plot(kind='kde')

In [ ]:
pd.DataFrame(scaled_frame2).plot(kind='hist', bins=30)

Using the MinMaxScaler method, we have again scaled the data into a uniform unit over all the columns. As you can see in the preceding table, the values of all the features have been converted into a uniform range of the same scale[0,1].

Ans: In Standardscaler, it assumes that data has normally distributed features and it has scaled them to zero mean and 1 standard deviation.

All the features are of the same scale after applying the scaler.

While in Minmaxscaler, it shrinks the data within the range of -1 to 1(if there are negative values) and responds well if standard deviation is small and is used when distribution is not Gaussian.This scaler is sensitive to outliers.


In Standard scaler, centered curves are there with no outliers and in minmax, outliers are there.

So here, we will continue with standard scaler.

In [ ]:
df_Con=df.drop(['Channel','Region'], axis=1)  # drop Categorical features

In [ ]:
df_Con.head()

In [ ]:
# Histogram
plt.figure(figsize=(10,8))
df_Con.plot(kind='hist', alpha=0.8,bins=60, subplots=True,layout=(3,2),legend=True,figsize=(12,10))

The histograms show an exponential decline in the number of orders for the respected products. hence this could be a cluster consisting of larger size companies with higher purchase quanitites for these particular items.

In [ ]:
plot1=sns.pairplot(df_Con, diag_kind='kde')

In [ ]:
mean_df=df.describe().loc['mean',:]
mean_df

Q. Find optimal number of features using RFECV and show the plot between Number of features 
selected vs Cross validation score (use channel as target variable)


In [ ]:
X = scaled_frame.drop(['Channel'], axis=1)
y = df['Channel'] # Channel has 2 values so we will use channel here from main table

# convert channel into binary values
y[y == 2] = 0
y[y == 1] = 1

y.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split

lr, knn = LinearRegression(), KNeighborsClassifier()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [ ]:
print("Fit raw features:")
print(" LR:", lr.fit(X_train, y_train).score(X_test, y_test))
print("KNN:", knn.fit(X_train, y_train).score(X_test, y_test))
print("GBC: ", GradientBoostingClassifier().fit(X_train, y_train).score(X_test, y_test))
print("RFC: ", RandomForestClassifier().fit(X_train, y_train).score(X_test, y_test))

In [ ]:
rf = RandomForestClassifier()
rfecv = RFECV(estimator=rf)
rfecv.fit(X, y)

# Plot number of features VS. cross-validation scores
plt.figure(figsize=(10,5))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.title("Optimal number of features : %d" % rfecv.n_features_)
plt.show()

Here we will use RFECV using GradientBoostingClassifier with pipleline

In [ ]:
model = GradientBoostingClassifier()
rfecv = RFECV(estimator=GradientBoostingClassifier())

Feature selection is an important task and it is crucial when the data in question has many features. The optimal number of features also leads to improved model accuracy. Obtaining the most important features and the number of optimal features can be obtained via feature importance or feature ranking.

## Recursive Feature Elimination

Instead of manually configuring the number of features(RFE), it would be very nice if we could automatically select them. This can be achieved via recursive feature elimination and cross-validation. This is done via the RFECV.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
import numpy as np

 Pipeline — since we’ll perform some cross-validation. It’s best practice in order to avoid data leakage.
    RepeatedStratifiedKFold — for repeated stratified cross-validation.
    cross_val_score — for evaluating the score on cross-validation.
    GradientBoostingClassifier — the estimator we’ll use.
    numpy — so that we can compute the mean of the scores.

In [ ]:
pipeline = Pipeline([('Feature Selection', rfecv), ('Model', model)])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
n_scores = cross_val_score(pipeline, X,y, scoring='accuracy', cv=cv, n_jobs = -1)
np.mean(n_scores)

Let’s fit the pipeline and then obtain the optimal number of features.

In [ ]:
pipeline.fit(X,y)

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)

In [ ]:
rfecv.support_

Once fitted, the following attributes can be obtained:
  
    ranking_ — the ranking of the features.
    n_features_ — the number of features that have been selected.
    support_ — an array that indicates whether or not a feature was selected.
    grid_scores_ — the scores obtained from cross-validation.

Lets put that into a dataframe and check the result

In [ ]:
rfecv_df = pd.DataFrame(rfecv.ranking_, index=X.columns, columns = ['Rank']).sort_values(by='Rank', ascending=True)
rfecv_df.head(10)

With the grid_scores_ we can plot a graph showing the cross-validated scores

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score ( nb of correct classification)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

We have concluded that Milk, Grocery, Frozen, Detergents_Paper and Delicassen are optimal features

Q. Implement KMeans Clustering for K=2 to K=15 and based on elbow method identify what is the optimum number of clusters

In [ ]:
from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score

In [ ]:
# First we need to convert our categorical features (region and channel) to dummy variable:
# df2 = pd.get_dummies(df)
# X1 = df2.iloc[:,:].values

In [ ]:
wcss = []
for i in range(2, 16):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(scaled_frame) # Standard scaler 
    wcss.append(kmeans.inertia_)
    cluster_labels = kmeans.labels_
    # silhouette score
#     silhouette_avg = silhouette_score(df, cluster_labels)
#     print("For n_clusters={0}, the silhouette score is {1}".format(i, silhouette_avg))
plt.plot(range(2, 16), wcss)
plt.title('The Elbow Method')
plt.xlabel('n: Number of clusters')
plt.ylabel('WCSS')
plt.show()

This graph looks like elbow and we have to(can) determine that elbow point.
Here the elbow point comes at around 6 and this our optimal number of clusters for the above data which we should choose.
If we look at the figure carefully after 6 when we go on increasing the number of cluster WCSS reduces slightly.

In [ ]:
kmeans = KMeans(n_clusters = 6,random_state = 111)
y_means = kmeans.fit(scaled_frame)

In [ ]:
y_means

In [ ]:
print('SSE: ', kmeans.inertia_)
print('\nCentroids: \n', kmeans.cluster_centers_)

In [ ]:
#count number of records in every cluster
pd.Series(kmeans.labels_).value_counts()

Q. Implement PCA with number of original features to answer how much variance is explained by first 2 components and by first 4 components and visualize the clusters in the data

In [ ]:
kmeans = KMeans(n_clusters=6, init='k-means++', max_iter=400, n_init=100, random_state=0)
y_means = kmeans.fit(scaled_frame)

In [ ]:
pca2 = PCA(n_components=2).fit(scaled_frame)
pca2d = pca2.transform(scaled_frame)

print("Explained variance is:",pca2.explained_variance_)
print("Explained variance ratio",pca2.explained_variance_ratio_)
print("Variance for 1st component is 38.75% & 2nd component is 22.37%")

From the above output, we can observe that the principal component 1 holds 38.75% of the information while the principal component 2 holds only 22.37% of the information.

In [ ]:
pca4 = PCA(n_components=4).fit(scaled_frame)
pca4d = pca4.transform(scaled_frame)

print("Explained variance is:",pca4.explained_variance_)
print("Explained variance ratio is:",pca4.explained_variance_ratio_)

plt.figure(figsize = (8,8))
sns.scatterplot(pca4d[:,0], pca4d[:,1], 
                hue=y_means.labels_, 
                palette='Set1',
                s=100, alpha=0.2).set_title('KMeans Clusters (4) Derived from Elbow Method', fontsize=15)
plt.legend()
plt.ylabel('PC2')
plt.xlabel('PC1')
plt.show()

From the above output, we can observe that the principal component 1 holds 38.75% of the information while the principal component 2 holds only 22.37% , 3 holds 12% and 4 holds 9%of the information.

Q. Implement XGBoost Classifier with 5 Fold CV and report the performance metrics

In [ ]:
X = df.drop('Channel', axis=1)

y = df['Channel']

In [ ]:
X.head()

In [ ]:
y.head()

y label contain values as 1 and 2
We will convert it into 0 and 1 for further analysis.

In [ ]:
# convert labels into binary values

y[y == 2] = 0

y[y == 1] = 1

In [ ]:
y.head()

We will convert the dataset into an optimized data structure called Dmatrix that XGBoost supports and gives it acclaimed performance and efficiency gains.

In [ ]:
# import sys
# !{sys.executable} -m pip install xgboost
# to install xgboost

## Train the XGBoost Classifier 

### k-fold Cross Validation using XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [ ]:
model = XGBClassifier(eval_metric='mlogloss')
kfold = KFold(n_splits=5)

In [ ]:
results = cross_val_score(model, X, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
Accuracy = results.mean()*100

In this kernel, I have implemented XGBoostclassifier with Python and Scikit-Learn to classify the customers from two different channels as Horeca (Hotel/Retail/Café) customers or Retail channel (nominal) customers.

The y labels contain values as 1 and 2. We have converted them into 0 and 1 for further analysis.

We have performed k-fold cross-validation with XGBoost classifier.
This example summarizes the performance of the default model configuration on the dataset including both the mean and standard deviation classification accuracy.

In [ ]:
# max_depth = 5, alpha = 10, n_estimators = 10)

model = XGBClassifier(eval_metric='mlogloss')
kfold = KFold(n_splits=5)

In [ ]:
PW = cross_val_score(model, X, y, cv=kfold, scoring='precision_weighted')
Rscore = cross_val_score(model, X, y, cv=kfold, scoring='recall_weighted')

In [ ]:
Recall = (round((Rscore.mean()*100),3))
Precision = (round((PW.mean()*100),3))

f1score=round(2*((Recall*Precision)/(Recall+Precision)),3)

In [ ]:
print("Accuracy= %f, Recall=%f, Precision=%f, f1score=%f" % (Accuracy,Recall,Precision,f1score))